In [ ]:
!pip install mlflow

In [ ]:
import mlflow

In [ ]:
mlflow.__version__

In [ ]:
mlflow.set_tracking_uri("http://app-mlflow-32adp:5000/")
mlflow.set_experiment("internal-experiment")
# Get Experiment Details
experiment = mlflow.get_experiment_by_name("internal-experiment")
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

In [ ]:
import tensorflow as tf
from mlflow import log_metric, log_param, log_artifacts, log_artifact
import os
from random import random, randint

print(tf.__version__)

mnist = tf.keras.datasets.mnist
with mlflow.start_run():
    mlflow.set_tag("mlflow.user", os.environ["PRIMEHUB_USER"])
    mlflow.tensorflow.autolog()

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=2)
    model.evaluate(x_test, y_test)
    model.save('tf-mlflow-model.h5')

    ## Random Log
    # Log a parameter (key-value pair)
    log_param("param1", randint(0, 100))

    # Log a metric; metrics can be updated throughout the run
    log_metric("foo", random())
    log_metric("foo", random() + 1)
    log_metric("foo", random() + 2)

    # Log an artifact (output file)
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("hello world!")
    log_artifacts("outputs")
    log_artifact('tf-mlflow-model.h5')